# Convolutional neural networks introduction

Convolutional neural networks are a specialized type of neural networks that use convolution in place of general matrix multiplication in at least one of their layers. They are one of the most important and most used tipes of neural networks, used in various applications.
Let's dive in and build a simple convolution neural network.

In [1]:
import torch
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as F  

from torch.utils.data import DataLoader

import torchvision.datasets as datasets
import torchvision.transforms as transforms

After the imports, let's define our hyperparameters that will be used:

In [2]:
in_channel = 1
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 5

Now, let's import our data:

In [3]:
train_dataset = datasets.MNIST('', train=True,
                               transform=transforms.ToTensor(), 
                               download=True)


test_dataset = datasets.MNIST('', train=False,
                               transform=transforms.ToTensor(), 
                               download=True)


train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)


Now, let's make a CNN class.
As a convolution layer, let's use ``nn.Conv2d`` that applies a 2D convolution over an input signal composed of several input planes.
As a pooling layer, let's use ``nn.MaxPool2d`` that applies a 2D max pooling.

In [4]:
class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=1,
            out_channels=8,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding=(1, 1),
        )
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.conv2 = nn.Conv2d(
            in_channels=8,
            out_channels=16,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding=(1, 1),
        )
        self.fc1 = nn.Linear(16 * 7 * 7, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)

        return x

Set the device:

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Initialize network:

In [6]:
model = CNN().to(device)

Let's calculate loss and specify the optimizer:

In [7]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

## Training

In [8]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = loss_function(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()
    print(loss)

tensor(0.0716, grad_fn=<NllLossBackward>)
tensor(0.0214, grad_fn=<NllLossBackward>)
tensor(0.1002, grad_fn=<NllLossBackward>)
tensor(0.0066, grad_fn=<NllLossBackward>)
tensor(0.0444, grad_fn=<NllLossBackward>)


Let's check accuracy on training & test to see how good our model is:


In [9]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Accuracy on training data:")
    else:
        print("Accuracy on test data:")

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()

    

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Accuracy on training data:
Got 59222 / 60000 with accuracy 98.70
Accuracy on test data:
Got 9839 / 10000 with accuracy 98.39
